# Before your start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [26]:
# Libraries
import pandas as pd
import numpy as np

# Challenge 1 - Index Functions

In this challenge, we will practice the advanced functions introduced in the lesson. However, before we start, we will load and evaluate our dataset. 

Load the dataset from Ironhack's database:
* db: `admissions predict`
* table: `admissions predict`

In [27]:
# your code here
data = pd.read_csv("Admission_Predict.csv") 

Remove trailing spaces at the end of the column names if there are any.


In [28]:
data.columns = data.columns.str.rstrip()
# Trim trailing space of column in pandas – rstrip()

Let's evaluate the dataset by looking at the `head` function.

In [29]:
# your code here
data.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Before beginning to work with this dataset and evaluating graduate admissions data, we will verify that there is no missing data in the dataset. Do this in the cell below.

In [30]:
# your code here
data.isnull().values.any()

False

Interestingly, there is a column that uniquely identifies the applicants. This column is the serial number column. Instead of having our own index, we should make this column our index. Do this in the cell below. Keep the column in the dataframe in addition to making it an index.

In [31]:
# your code here
data.set_index('Serial No.', inplace=True)

In [32]:
data.head()

,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
Serial No.,,,,,,,,
1,337,118,4,4.5,4.5,9.65,1,0.92
2,316,104,3,3.0,3.5,8.00,1,0.72
3,322,110,3,3.5,2.5,8.67,1,0.80
4,314,103,2,2.0,3.0,8.21,0,0.65
5,330,115,5,4.5,3.0,9.34,1,0.90


Turns out that `GRE Score` and `CGPA` also uniquely identify the data. Show this in the cell below.

In [33]:
# your code here
print(data.set_index(['GRE Score']).index.is_unique)

print(data.set_index(['CGPA']).index.is_unique)

print(data.set_index(['GRE Score','CGPA']).index.is_unique)

False
False
True


Replace the index with an index comprised of two columns - `GRE Score` and `CGPA`. Remove the columns from the dataset as well. Make sure to use `inplace=True`.

Now change the index back to a sequence starting at zero using the `reset_index` function. Make sure to use `inplace=True`.

In [34]:
# your code here
data.set_index(['GRE Score','CGPA'], inplace=True)

In [35]:
data.head()

,,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
GRE Score,CGPA,,,,,,
337,9.65,118,4,4.5,4.5,1,0.92
316,8.00,104,3,3.0,3.5,1,0.72
322,8.67,110,3,3.5,2.5,1,0.80
314,8.21,103,2,2.0,3.0,0,0.65
330,9.34,115,5,4.5,3.0,1,0.90


In [36]:
data.reset_index(inplace=True)

In [37]:
data.head()

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,118,4,4.5,4.5,1,0.92
1,316,8.00,104,3,3.0,3.5,1,0.72
2,322,8.67,110,3,3.5,2.5,1,0.80
3,314,8.21,103,2,2.0,3.0,0,0.65
4,330,9.34,115,5,4.5,3.0,1,0.90


# Challenge 2 - Advanced Functions

In this part of the lab, we would like to test complex conditions on the entire dataset at once. Let's start by finding the number of rows where the CGPA is higher than 9 and the student has conducted research.

In [38]:
# your code here
rslt_df = data[(data['CGPA'] > 9) & data['Research'] == 1]

rslt_df.head()

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,118,4,4.5,4.5,1,0.92
4,330,9.34,115,5,4.5,3.0,1,0.90
10,328,9.10,112,4,4.0,4.5,1,0.78
19,328,9.50,116,5,5.0,5.0,1,0.94
20,334,9.70,119,5,5.0,4.5,1,0.95


Now return all the rows where the CGPA is greater than 9 and the SOP score is less than 3.5. Find the mean chance of admit for these applicants.

In [39]:
# your code here
rslt_df = data[(data['CGPA'] > 9) & data['SOP'] < 3.5]

rslt_df.head()

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit
0,337,9.65,118,4,4.5,4.5,1,0.92
1,316,8.00,104,3,3.0,3.5,1,0.72
2,322,8.67,110,3,3.5,2.5,1,0.80
3,314,8.21,103,2,2.0,3.0,0,0.65
4,330,9.34,115,5,4.5,3.0,1,0.90


In [40]:
rslt_df['Chance of Admit'].mean()

0.7239220779220773

We would like to create a deciding factor column for each student. We standardize several columns and then pick the most important factor from a lookup table. If the standardized value is above 0.8, the student will be accepted.

We will start by creating a standardized column for `CGPA`, `GRE Score` and `LOR`. We will name these columns `CGPA_std`, `GRE_std`, and `LOR_std` respecively.

Recall that standardizing a column is done by subtracting the mean of the column from all observations in the column and then dividing each observation in the column by the column's standard deviation.

In the cell below, write a standardization function (a function that takes a column as input and returns a standardized column as output). Make sure to use the `numpy` versions of mean and standard deviation.

In [41]:
def standardize(col):
    """
    This function takes a column from a dataframe 
    and returns a standardized column by subtracting the column's mean
    and dividing by the column's standard deviation.
    """
    # your code here
    the_mean = np.mean(col)  
    the_std = np.std(col)
    
    return (col - the_mean) /the_std

Now create the standardized columns `CGPA_std`, `GRE_std`, and `LOR_std` and add them to the `admissions` dataframe.

In [42]:
# your code here
data["CGPA_std"] = standardize(data["CGPA"])
data["GRE_std"] = standardize(data["GRE Score"])
data["LOR_std"] = standardize(data["LOR"])

data.head()

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std
0,337,9.65,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197
1,316,8.00,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840
2,322,8.67,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517
3,314,8.21,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338
4,330,9.34,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338


We will generate the decision choice at random using the code below. Please run the cell.

In [3]:
# Libraries
from random import choices

In [6]:
std_columns = ['CGPA_std', 'GRE_std', 'LOR_std']

decision_choice = choices(std_columns, k=data.shape[0])

data.head()

,Serial No.,GRE Score,TOEFL Score,University Rating,SOP,LOR,CGPA,Research,Chance of Admit
0,1,337,118,4,4.5,4.5,9.65,1,0.92
1,2,316,104,3,3.0,3.5,8.00,1,0.72
2,3,322,110,3,3.5,2.5,8.67,1,0.80
3,4,314,103,2,2.0,3.0,8.21,0,0.65
4,5,330,115,5,4.5,3.0,9.34,1,0.90


Now create the deciding column using the `lookup` function. The lookup column is `decision_choice` found above. Call the column resulting from the lookup function `deciding_column` and add it to the `admissions` dataframe.

In [44]:
# your code here
#The lookup() function returns label-based "fancy indexing" function for DataFrame.
data['deciding_column'] = data.lookup(data.index, decision_choice)
data.head()

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column
0,337,9.65,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.755663
1,316,8.00,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.063450
2,322,8.67,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,-1.039517
3,314,8.21,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338
4,330,9.34,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,1.149292


Create a column called `decision` in the `admissions` dataframe. Assign 1 to this column if the value of `deciding_column` is greater than 0.8 and 0 otherwise.

In [45]:
# your code here
#where condition [x,y]
data['decision'] = np.where(data['deciding_column'] > 0.8, '1', '0')
data.head()

,GRE Score,CGPA,TOEFL Score,University Rating,SOP,LOR,Research,Chance of Admit,CGPA_std,GRE_std,LOR_std,deciding_column,decision
0,337,9.65,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.755663,1
1,316,8.00,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.063450,0
2,322,8.67,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,-1.039517,0
3,314,8.21,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338,0
4,330,9.34,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,1.149292,1


How many applicants will be accepted to the program using the decision column? Compute the result below.

In [46]:
# your code here
data['decision'].value_counts()
#decision 1 in 85 cases

0    300
1     85
Name: decision, dtype: int64

# Challenge 3 - Method Chaining

To increase our coding efficiency, let's make a number of changes to our dataframe in one line of code.

In the cell below, remove all non character symbols from the column names, replace all spaces in column names to underscores, and change all upper case characters to lower case. Assign these new column values to `admissions.columns`.

In [47]:
# your code here
data.columns = data.columns.str.replace('[^\w\s]','').str.replace('\s','_').str.lower()
data.head()

,gre_score,cgpa,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision
0,337,9.65,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.755663,1
1,316,8.00,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.063450,0
2,322,8.67,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,-1.039517,0
3,314,8.21,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338,0
4,330,9.34,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,1.149292,1


In the cell below, give all student with a university rating of 4 or higher a 10 point boost on their GRE score and split the column into 4 bins using the `cut` function. Assign this new score to the variable `adjusted_gre`.

In [ ]:
# your code here

In [48]:
data['adjusted_gre'] = np.where(data['university_rating'] >= 4, data['gre_score'] + 10, data['gre_score'])

In [49]:
data.head(10)

,gre_score,cgpa,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre
0,337,9.65,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.755663,1,347
1,316,8.00,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.063450,0,316
2,322,8.67,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,-1.039517,0,322
3,314,8.21,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338,0,314
4,330,9.34,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,1.149292,1,340
5,321,8.20,109,3,3.0,4.0,1,0.75,-0.660055,0.369672,0.635019,0.369672,0,321
6,308,7.90,101,2,3.0,4.0,0,0.68,-1.158723,-0.756445,0.635019,-0.756445,0,308
7,302,8.00,102,1,2.0,1.5,0,0.50,-0.992501,-1.276191,-2.155874,-2.155874,0,302
8,323,8.60,108,3,3.5,3.0,0,0.45,0.004836,0.542921,-0.481338,-0.481338,0,323
9,327,9.00,111,4,4.0,4.5,1,0.84,0.669726,0.889419,1.193197,0.669726,0,337


In [56]:
data['adjusted_gre'].value_counts()

312    15
314    13
317    13
316    12
313    12
311    11
318    11
315    11
320    11
324    11
300    10
308     9
299     8
334     8
319     8
332     8
321     8
322     8
307     7
326     7
301     7
298     7
350     7
344     7
341     7
331     7
339     7
337     7
335     6
305     6
306     6
338     6
310     6
304     6
323     6
325     5
302     5
346     5
296     5
327     5
309     5
340     5
342     5
343     4
345     4
295     4
297     4
348     4
330     4
303     4
333     4
349     3
329     3
336     3
294     2
347     1
293     1
290     1
Name: adjusted_gre, dtype: int64

In [61]:
data['adjusted_gre'] = pd.cut(x=data['adjusted_gre'], bins=[250, 280, 320, 350])


TypeError: unorderable types: Interval() > int()

In [60]:
data.head()

,gre_score,cgpa,toefl_score,university_rating,sop,lor,research,chance_of_admit,cgpa_std,gre_std,lor_std,deciding_column,decision,adjusted_gre
0,337,9.65,118,4,4.5,4.5,1,0.92,1.750174,1.755663,1.193197,1.755663,1,"(320, 350]"
1,316,8.00,104,3,3.0,3.5,1,0.72,-0.992501,-0.063450,0.076840,-0.063450,0,"(280, 320]"
2,322,8.67,110,3,3.5,2.5,1,0.80,0.121191,0.456297,-1.039517,-1.039517,0,"(320, 350]"
3,314,8.21,103,2,2.0,3.0,0,0.65,-0.643433,-0.236698,-0.481338,-0.481338,0,"(280, 320]"
4,330,9.34,115,5,4.5,3.0,1,0.90,1.234884,1.149292,-0.481338,1.149292,1,"(320, 350]"


In [62]:
print(data['adjusted_gre'].unique())

[(320, 350], (280, 320]]
Categories (2, interval[int64]): [(280, 320] < (320, 350]]
